In [144]:
# Imports
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import time

In [6]:
NAMES_MALE = "../data/имена_српска_мушка.txt"
NAMES_FEMALE = "../data/имена_српска_женска.txt"

RESULTS_MALE = "../results/имена_српска_мушка_makemore_2003.txt"
RESULTS_FEMALE = "../results/имена_српска_женска_makemore_2003.txt"


In [138]:
# CONFIG VARIABLES
NAMES_FILE = NAMES_MALE
block_size = 3
embedding_dimensions = 10
neurons = 200
batch_size = 64
lr = 0.01

In [60]:
with open(NAMES_FILE, 'r', encoding='utf-8') as f:
    words = f.read().splitlines()
# Convert the first letter of each word to lowercase
words = [word[0].lower() + word[1:] if word else word for word in words]
words[:5]


['бадач', 'баја', 'бајо', 'бајан', 'бајаш']

In [61]:
len(words)

3646

In [62]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(stoi)


{'а': 1, 'б': 2, 'в': 3, 'г': 4, 'д': 5, 'е': 6, 'ж': 7, 'з': 8, 'и': 9, 'к': 10, 'л': 11, 'м': 12, 'н': 13, 'о': 14, 'п': 15, 'р': 16, 'с': 17, 'т': 18, 'у': 19, 'х': 20, 'ц': 21, 'ч': 22, 'ш': 23, 'ђ': 24, 'ј': 25, 'љ': 26, 'њ': 27, 'ћ': 28, '.': 0}


## Building the Dataset:

```X - input of the Neural Network, cotext window of the size: block_size,  Context that is used as an input, if block_size 3, input will be 3 words and guess the third one```

```Y - Next word is the label (it is what model is needs to guess)```

In [63]:
# Building the Dataset
def build_dataset(words, block_size, stoi):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context) # Context that is used as an input, if block_size 3, input will be 3 words and guess the third one
            Y.append(ix)  # Next word is the label (it is what model is needs to guess)
            context = context[1:] + [ix]  # crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

In [64]:
# Split the data
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1], block_size, stoi=stoi)
Xdev, Ydev = build_dataset(words[n1:n2], block_size, stoi=stoi)
Xte, Yte = build_dataset(words[n2:], block_size, stoi=stoi)

len(Xtr), len(Ytr), len(Xdev), len(Ydev), len(Xte), len(Yte)

(21777, 21777, 2705, 2705, 2721, 2721)

In [69]:
num_of_letters = len(stoi.keys()) # Number of tokens (in this case number of letters in Azbuka(30) + Special Char '.' = total max (31)
# INIT PARAMS
C = torch.randn((num_of_letters, embedding_dimensions))
W1 = torch.randn((block_size * embedding_dimensions, neurons))
b1 = torch.randn(neurons)
W2 = torch.randn((neurons, num_of_letters))
b2 = torch.randn(num_of_letters)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

print(f"Number of tokens: {num_of_letters}")


Number of tokens: 29


In [86]:
C.shape

torch.Size([29, 10])

In [89]:
W1.shape, b1.shape, W2.shape, b2.shape

(torch.Size([30, 200]),
 torch.Size([200]),
 torch.Size([200, 29]),
 torch.Size([29]))

In [112]:
X, Y = Xtr, Ytr


In [141]:
for _ in range(40000):
    # Random MiniBatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))


    # Forward Pass
    emb = C[X[ix]]  # (N, block_size(3), vectors_representation(10))
    h = torch.tanh(emb.view(-1, block_size * embedding_dimensions) @ W1 + b1)  # (N, 200)
    logits = h @ W2 + b2  # (N, 29)
    loss = F.cross_entropy(logits, Y[ix])

    # Backwards Pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # Update
    for p in parameters:
        p.data += -lr * p.grad

print(loss.item())

1.4004154205322266


In [104]:
# Manual Loss

# counts = logits.exp()
# probs = counts / counts.sum(1, keepdim=True)
# loss_manual = -probs[torch.arange(len(X)), Y].log().mean()

(torch.Size([21777, 29]), torch.Size([21777]))

In [142]:
emb = C[X]  # (N, block_size(3), vectors_representation(10))
h = torch.tanh(emb.view(-1, block_size * embedding_dimensions) @ W1 + b1)  # (N, 200)
logits = h @ W2 + b2  # (N, 29)
loss = F.cross_entropy(logits, Y)
loss


tensor(1.3373, grad_fn=<NllLossBackward0>)

In [159]:
num_of_names = 5
# Sample from the model
random_seed = int(time.time() * 1000) % (2**32)  # Ограничавамо на 32-битни опсег
g = torch.Generator().manual_seed(random_seed)

for _ in range(num_of_names):
    out = []
    context = [0] * block_size  # Initialize with all ...
    while True:
        emb = C[torch.tensor([context])]  # (1, block_size, d)
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        if ix == 0:
            break
        out.append(ix)

    # Rekonstruiši ime i promeni prvo slovo u veliko
    name = ''.join(itos[i] for i in out)
    name = name.capitalize()  # Pretvori prvo slovo u veliko

    print(name)

Југочај
Слован
Веселисав
Лета
Рујко
